In [4]:
import csv


In [6]:
with open('conv1_speads.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        print(row)

['speaker0_F', '0.00000', '1.09000']
['speaker0_F', '1.65000', '2.60000']
['speaker1_M', '3.07000', '4.56000']
['speaker2_M', '7.35000', '20.74000']
['speaker0_F', '20.74000', '23.20000']
['speaker3_F', '23.20000', '24.73000']
['speaker0_F', '24.73000', '27.33000']
['speaker4_F', '29.98000', '41.44000']
['speaker3_F', '41.44000', '43.91000']
['speaker1_M', '46.53000', '75.32000']
['speaker0_F', '76.14000', '78.75000']
['speaker0_F', '79.98000', '90.58000']
['speaker4_F', '90.58000', '97.64000']
['speaker3_F', '97.64000', '129.35001']
['speaker4_F', '129.35001', '134.85001']
['speaker3_F', '134.85001', '152.25999']
['speaker4_F', '152.25999', '156.28000']


In [9]:
def preprocess_raw_speads_output():
    with open('conv1_processed.tsv', 'w') as dest:
        dest.write("ID\tGender\tStart\tEnd\n")
        with open('conv1_speads.tsv', 'r') as source:
            reader = csv.reader(source, delimiter='\t')
            for row in reader:
                dest.write('{}\t{}\t{}\t{}\n'.format(row[0][:-2], row[0][-1:], row[1], row[2]))

In [10]:
preprocess_raw_speads_output()